<a href="https://colab.research.google.com/github/GhazalehKeyvani/Avina/blob/main/MLCrashCourse/fa_Validation_and_Test_Sets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   جدا کردن دیتاست داده های آموزشی ، اعتبار سنجی و تست

*   آنالیز تفاوت داده های آموزشی و اعتبارسنجی
*    تست مدل آموزش دیده و پدیده ی overfitting


*   تشخیص و تعمیر مشکلات آموزشی مسئله



مجموعه داده
همانند تمرین قبلی، این تمرین از مجموعه داده مسکن کالیفرنیا برای پیش‌بینی مقدار_خانه_متوسط در سطح بلوک شهر استفاده می‌کند. مانند بسیاری از مجموعه داده های "مشهور"، مجموعه داده مسکن کالیفرنیا در واقع از دو مجموعه داده جداگانه تشکیل شده است که هر کدام در فایل های csv. جداگانه زندگی می کنند:

مجموعه آموزشی در california_housing_train.csv است.
مجموعه تست در california_housing_test.csv است.
با تقسیم مجموعه آموزشی دانلود شده به دو قسمت، مجموعه اعتبارسنجی را ایجاد خواهید کرد:

یک مجموعه آموزشی کوچکتر
یک مجموعه اعتبار سنجی

#@title Import modules
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [ ]:
#@title Import modules
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

##بارگیری دیتاست ها از اینترنت
کد زیر دیتاست ها را از اینترنت بارگیری میکند

* `train_df`
* `test_df`

In [ ]:
train_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
test_df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")

##مقیاس مقادیر برچسب ها


In [ ]:
scale_factor = 1000.0

train_df["median_house_value"] /= scale_factor

test_df["median_house_value"] /= scale_factor

In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))



  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error.
  model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model

def train_model(model, df, feature, label, my_epochs,
                my_batch_size=None, my_validation_split=0.1):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=my_validation_split)

  # Gather the model's trained weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the
  # rest of history.
  epochs = history.epoch

  # Isolate the root mean squared error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history

print("Defined the build_model and train_model functions.")

##تعریف توابع رسم نمودار

رسم نمودار ها با تابع `plot_the_loss_curve` و تفاوت loss , epochs در مموعه های آموزشی و اعتبار سنجی

In [ ]:
#@title Define the plotting function

def plot_the_loss_curve(epochs, mae_training, mae_validation):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs[1:], mae_training[1:], label="Training Loss")
  plt.plot(epochs[1:], mae_validation[1:], label="Validation Loss")
  plt.legend()


  plt.plot(epochs[1:], mae_training[1:], label="Training Loss")
  plt.plot(epochs[1:], mae_validation[1:], label="Validation Loss")
  plt.legend()

  top_of_y_axis = highest_loss + (delta * 0.05)
  bottom_of_y_axis = lowest_loss - (delta * 0.05)

  plt.ylim([bottom_of_y_axis, top_of_y_axis])
  plt.show()

print("Defined the plot_the_loss_curve function.")

##تمرین 1 :کسب تجربه در جداسازی با داده های اعتبار سنجی

در سلول کد زیر، متغیری به نام `validation_split` را مشاهده خواهید کرد که آن را در 0.2 مقداردهی اولیه کرده ایم. متغیر `validation_split` نسبت مجموعه آموزشی اصلی را مشخص می کند که به عنوان مجموعه اعتبار سنجی عمل می کند. مجموعه آموزشی اصلی شامل 17000 نمونه است. بنابراین، یک `validation_split` از 0.2 به این معنی است که:


*   validation set --> 17000 * 0.2 = 3400
*   training set --> 17000 * 0.8 = 13600



*   the training set
*   the validation set

اگر داده‌های مجموعه آموزشی مشابه داده‌های مجموعه اعتبارسنجی باشد، دو منحنی ضرر و مقادیر تلفات نهایی باید تقریباً یکسان باشند. با این حال، منحنی‌های loss و مقادیر loss نهایی تقریباً یکسان **نیستند**. هوم، عجیب است.

با دو یا سه مقدار مختلف validation_split آزمایش کنید. آیا مقادیر مختلف validation_split مشکل را برطرف می کند؟

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.1
epoch = 17
batch_size = 100

# Split the original training set into a reduced training set and a
# validation set.
validation_split = 0.2

# Identify the feature and the label.
feature = "median_income"
lable = "median_house_value"

# Invoke the functions to build and train the model.

my_model = build_model(larning_rate)

epochs, rmse, history = train_model(my_model, train_df, feature, lable,
                                    epochs, batch_size, validation_split
                                    )

plot_the_loss_curve(epochs, history["root_mean_squared_error"],
                    history["val_root_mean_squared_error"]
                    )

تمرین 2 :
مهم نیست که چطور داده های آموزشی و اعتبار سنجی را جداسازی می کنیم
، از نظر معنایی منحنی lossآنها متفاوت است

داده ها در ست آموزشی به اندازه کافی مشابه داده های ست اعتبار سنجی نیستند

وظیفه ی شما اینه که مشخص کنید چرا منحنی loss آنها مشابه نیستند



*   سلول کد قبلی شامل:
  

  *   a reduced training set (the original training set - the validation set)
  *   the validation set


*   پانداس متد  head را برای خروجی شامل 5 ردیف اول دیتا فریم به میدهد



In [ ]:
train_df.head(1000)

تمرین 3: حل مسئله
برای رفع این مشکل، نمونه ها را shuffle
 در مجموعه ی آموزشی قبل از جداسازی یا درهمسازی می کنیم.



1.   درهم سازی داده در مجموعه داده ی آموزشی قبل از صدا زدن train_model

```
shuffled_train_test = train_df.reindex(np.random.permutation(train_df.index))
```


2.   و به عنوان دومین آرگومان به صورت زیر shuffle_train_model را به train_model بدهید


```
epochs,rmse, history = train_model(my_model, shffled_train_df, my_feature,my_lable, epochs, batch_size, validation_split)
```





In [ ]:
#@title Double-click to view the complete implementation.

# The following variables are the hyperparameters.
learning_rate = 0.08
epochs = 70
batch_size = 100

# Split the original training set into a reduced training set and a
# validation set.
validation_split = 0.2


# Identify the feature and the label.
my_feature = "median_income"    # the median income on a specific city block.
my_label = "median_house_value" # the median house value on a specific city block.
# That is, you're going to create a model that predicts house value based
# solely on the neighborhood's median income.

# Shuffle the examples.
shuffled_train_df = train_df.reindex(np.random.permutation(train_df.index))

# Invoke the functions to build and train the model. Train on the shuffled
# training set.
my_model = build_model(learning_rate)
epochs, rmse, history = train_model(my_model, shuffled_train_df, my_feature,
                                    my_label, epochs, batch_size,
                                    validation_split)

plot_the_loss_curve(epochs, history["root_mean_squared_error"],
                            history["val_root_mean_squared_error"])

برای پاسخ دادن به سوالات زیر با validation_split آزمایش کنید:

*  با اختلاط مجموعه آموزشی، آیا ضرر نهایی مجموعه آموزشی به ضرر نهایی مجموعه اعتبارسنجی نزدیکتر است؟
*   در چه محدوده ای از مقادیر validation_split، مقادیر تلفات نهایی برای مجموعه آموزشی و مجموعه اعتبارسنجی به طور معنی داری از هم جدا می شوند؟ چرا؟

In [ ]:
#@title برای پاسخ به سوالات دوبار کلیک کنید

# بله، پس از به هم زدن مجموعه آموزشی اصلی،
# باخت نهایی برای مجموعه تمرینی و
# مجموعه اعتبارسنجی بسیار نزدیکتر شده است.

# اگر validation_split < 0.15،
# مقادیر خطای نهایی برای مجموعه آموزشی و
# مجموعه اعتبار سنجی به طور معناداری واگرا می شود. زیرا ،
# مجموعه اعتبارسنجی دیگر شامل نمونه های کافی نیست.

##وظیفه 4: از مجموعه داده تست برای ارزیابی عملکرد مدل خود استفاده کنید
مجموعه تست معمولا به عنوان قاضی نهایی کیفیت یک مدل عمل می کند. مجموعه آزمون می تواند به عنوان یک قاضی بی طرف عمل کند زیرا از نمونه های آن در آموزش مدل استفاده نشده است. سلول کد زیر را برای ارزیابی مدل با مجموعه تست اجرا کنید:

In [ ]:
x_test = test_df[my_feature]
y_test = test_df[my_label]

results = my_model.evaluate(x_test,x_test,batch_size=batch_size)

ریشه میانگین مربعات خطای مدل را هنگام ارزیابی هر یک از سه مجموعه داده مقایسه کنید:

* مجموعه آموزشی: به دنبال `root_mean_squared_error` در دوره آموزشی نهایی بگردید.
* مجموعه اعتبارسنجی: به دنبال `val_root_mean_squared_error` در دوره آموزشی نهایی بگردید.
* مجموعه آزمایشی: سلول کد قبلی را اجرا کنید و `root_mean_squared_error` را بررسی کنید.

در حالت ایده آل، ریشه میانگین مربعات خطای هر سه مجموعه باید مشابه باشد. هستند آنها؟

In [ ]:
#@title برای پاسخ دوبار کلیک کنید

# در آزمایشات ما، بله، مقادیر rmse
# به اندازه کافی شبیه بودند.